<a href="https://colab.research.google.com/github/JosephTeja/Analytics-Projects/blob/master/TRAIL_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/FurkanOM/tf-ssd.git

Cloning into 'tf-ssd'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 277 (delta 187), reused 175 (delta 91), pack-reused 0
Receiving objects: 100% (277/277), 79.89 KiB | 3.63 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [ ]:
%cd /content/drive/My Drive/SSD

/content/drive/My Drive/SSD


In [ ]:
%cd /content/drive/My Drive/SSD/tf-ssd

/content/drive/My Drive/SSD/tf-ssd


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam
import augmentation
from ssd_loss import CustomLoss
from utils import bbox_utils, data_utils, io_utils, train_utils


batch_size = 16
epochs = 2
load_weights = True
with_voc_2012 = False
backbone = "mobilenet_v2"
io_utils.is_valid_backbone(backbone)
#
if backbone == "mobilenet_v2":
    from models.ssd_mobilenet_v2 import get_model, init_model
else:
    from models.ssd_vgg16 import get_model, init_model
#
hyper_params = train_utils.get_hyper_params(backbone)
#
train_data, info = data_utils.get_dataset("voc/2007", "train+validation")
val_data, _ = data_utils.get_dataset("voc/2007", "test")
train_total_items = data_utils.get_total_item_size(info, "train+validation")
val_total_items = data_utils.get_total_item_size(info, "test")

if with_voc_2012:
    voc_2012_data, voc_2012_info = data_utils.get_dataset("voc/2012", "train+validation")
    voc_2012_total_items = data_utils.get_total_item_size(voc_2012_info, "train+validation")
    train_total_items += voc_2012_total_items
    train_data = train_data.concatenate(voc_2012_data)

labels = data_utils.get_labels(info)
labels = ["bg"] + labels
hyper_params["total_labels"] = len(labels)
img_size = hyper_params["img_size"]

train_data = train_data.map(lambda x : data_utils.preprocessing(x, img_size, img_size, augmentation.apply))
val_data = val_data.map(lambda x : data_utils.preprocessing(x, img_size, img_size))

data_shapes = data_utils.get_data_shapes()
padding_values = data_utils.get_padding_values()
train_data = train_data.shuffle(batch_size*4).padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)
val_data = val_data.padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)
#
ssd_model = get_model(hyper_params)
ssd_custom_losses = CustomLoss(hyper_params["neg_pos_ratio"], hyper_params["loc_loss_alpha"])
ssd_model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss=[ssd_custom_losses.loc_loss_fn, ssd_custom_losses.conf_loss_fn])
init_model(ssd_model)
#
ssd_model_path = io_utils.get_model_path(backbone)
if load_weights:
    ssd_model.load_weights("/content/drive/My Drive/SSD/tf-ssd/ssd_saved_model.h5")
ssd_log_path = io_utils.get_log_path(backbone)
# We calculate prior boxes for one time and use it for all operations because of the all images are the same sizes
prior_boxes = bbox_utils.generate_prior_boxes(hyper_params["feature_map_shapes"], hyper_params["aspect_ratios"])
ssd_train_feed = train_utils.generator(train_data, prior_boxes, hyper_params)
ssd_val_feed = train_utils.generator(val_data, prior_boxes, hyper_params)

step_size_train = train_utils.get_step_size(train_total_items, batch_size)
step_size_val = train_utils.get_step_size(val_total_items, batch_size)


In [ ]:
train_total_items/16

313.1875

In [ ]:
ssd_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 301, 301, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 150, 150, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 150, 150, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
ssd_model.fit(ssd_train_feed,
              steps_per_epoch=310,
              epochs=5)
ssd_model.save_weights("ssd_saved_model.h5")
print("Saved model to disk")

Epoch 1/5
310/310 [==============================] - 87s 279ms/step - loss: 6.7406 - loc_loss: 2.0410 - conf_loss: 4.6996
Epoch 2/5
310/310 [==============================] - 85s 274ms/step - loss: 5.7002 - loc_loss: 1.6988 - conf_loss: 4.0014
Epoch 3/5
310/310 [==============================] - 85s 275ms/step - loss: 5.5327 - loc_loss: 1.6562 - conf_loss: 3.8765
Epoch 4/5
310/310 [==============================] - 87s 280ms/step - loss: 5.3335 - loc_loss: 1.5680 - conf_loss: 3.7655
Epoch 5/5
310/310 [==============================] - 85s 273ms/step - loss: 5.0168 - loc_loss: 1.4789 - conf_loss: 3.5380
Saved model to disk


In [ ]:
ssd_model.fit(ssd_val_feed,
              steps_per_epoch=310,
              epochs=5)
ssd_model.save_weights("ssd_saved_model_val.h5")
print("Saved model to disk")

  2/310 [..............................] - ETA: 1:10 - loss: 4.6309 - loc_loss: 1.3621 - conf_loss: 3.2687WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0643s vs `on_train_batch_end` time: 0.1694s). Check your callbacks.


310/310 [==============================] - 73s 236ms/step - loss: 4.8546 - loc_loss: 1.4409 - conf_loss: 3.4137
Epoch 2/5
310/310 [==============================] - 71s 230ms/step - loss: 4.1255 - loc_loss: 1.2295 - conf_loss: 2.8960
Epoch 3/5
310/310 [==============================] - 72s 233ms/step - loss: 3.7299 - loc_loss: 1.1122 - conf_loss: 2.6177
Epoch 4/5
310/310 [==============================] - 72s 233ms/step - loss: 3.4270 - loc_loss: 1.0366 - conf_loss: 2.3904
Epoch 5/5
310/310 [==============================] - 71s 230ms/step - loss: 3.1184 - loc_loss: 0.9583 - conf_loss: 2.1601
Saved model to disk


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
!python predictor.py --backbone mobilenet_v2

2020-11-11 08:20:51.631962: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-11 08:20:53.557927: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-11 08:20:53.562367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 08:20:53.562838: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-11 08:20:53.562874: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-11 08:20:53.564787: I tensorflow/stream_executor/platform/default

In [ ]:
!python predictor.py --backbone mobilenet_v2

2020-11-11 08:31:23.806425: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-11 08:31:25.756735: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-11 08:31:25.761199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 08:31:25.761676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-11 08:31:25.761713: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-11 08:31:25.763276: I tensorflow/stream_executor/platform/default

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

#path of the directory where you want to save your model
frozen_out_path = '/content/drive/My Drive/SSD/'
# name of the .pb file
frozen_graph_filename = 'SSD_Mobilenet_frozen'
# Convert Keras model to ConcreteFunction

full_model = tf.function(lambda x: ssd_model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(ssd_model.inputs[0].shape, ssd_model.inputs[0].dtype))

# Get frozen ConcreteFunction

frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]

print("-" * 60)
print("Frozen model layers: ")

for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph to disk

tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)

# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

------------------------------------------------------------
Frozen model layers: 
x
functional_5/Conv1_pad/Pad/paddings
functional_5/Conv1_pad/Pad
functional_5/Conv1/Conv2D/ReadVariableOp/resource
functional_5/Conv1/Conv2D/ReadVariableOp
functional_5/Conv1/Conv2D
functional_5/bn_Conv1/ReadVariableOp/resource
functional_5/bn_Conv1/ReadVariableOp
functional_5/bn_Conv1/ReadVariableOp_1/resource
functional_5/bn_Conv1/ReadVariableOp_1
functional_5/bn_Conv1/FusedBatchNormV3/ReadVariableOp/resource
functional_5/bn_Conv1/FusedBatchNormV3/ReadVariableOp
functional_5/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1/resource
functional_5/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1
functional_5/bn_Conv1/FusedBatchNormV3
functional_5/Conv1_relu/Relu6
functional_5/expanded_conv_depthwise/depthwise/ReadVariableOp/resource
functional_5/expanded_conv_depthwise/depthwise/ReadVariableOp
functional_5/expanded_conv_depthwise/depthwise
functional_5/expanded_conv_depthwise_BN/ReadVariableOp/resource
functional_5/e

'/content/drive/My Drive/SSD/SSD_Mobilenet_frozen.pbtxt'

In [ ]:
# inputs:  ['dense_input']
print('inputs: ', [input.op.name for input in ssd_model.inputs])

# outputs:  ['dense_4/Sigmoid']
print('outputs: ', [output.op.name for output in ssd_model.outputs])

inputs:  ['input_3']
outputs:  ['loc/concat_2', 'conf/truediv_2']
